INSTALL
- FAISS - 
- Sent2Vec

In [2]:
import numpy as np
import faiss   

In [3]:
import os
import time
name = "OPUS-bert-"+ str(time.strftime("%Y%m%d-%H%M"))

!mkdir $name
path = os.getcwd() + "/" + name
os.chdir(path)

!pwd


os.chdir(path)


# import os
# os.chdir("/home/lw/clc_fairseq/TED-20210121-1335")

/home/chinh/clc_data/dataset/OPUS-bert-20210517-2115


In [4]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/opus-100-corpus.zip
!mkdir data_bin
!unzip 'opus-100-corpus.zip' -d 'data_bin'

--2021-05-17 21:15:47--  https://github.com/hoangtrungchinh/clc_data/raw/master/opus-100-corpus.zip
Resolving github.com (github.com)... 52.74.223.119
Connecting to github.com (github.com)|52.74.223.119|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/opus-100-corpus.zip [following]
--2021-05-17 21:15:48--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/opus-100-corpus.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30690851 (29M) [application/zip]
Saving to: ‘opus-100-corpus.zip’

opus-100-corpus.zip 100%[===================>]  29,27M  71,2KB/s    in 10m 39s 

2021-05-17 21:26:30 (46,9 KB/s) - ‘opus-100-corpus.zip’ saved [30690851/30690851]

In [5]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
vi_files = sorted(glob.glob("data_bin/*.vi"))
print(en_files)
print(vi_files)

lst_en = []
lst_vi = []

start = time.time()

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line)

for file in vi_files:
  with open(file) as file_in:
    for line in file_in:
      lst_vi.append(line)

print(len(lst_en))
print(len(lst_vi))

['data_bin/opus.en-vi-dev.en', 'data_bin/opus.en-vi-test.en', 'data_bin/opus.en-vi-train.en']
['data_bin/opus.en-vi-dev.vi', 'data_bin/opus.en-vi-test.vi', 'data_bin/opus.en-vi-train.vi']
1004000
1004000


In [6]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, vi_train, vi_test_valid =  train_test_split(lst_en, lst_vi, test_size=0.2, random_state=123)
en_valid, en_test, vi_valid, vi_test = train_test_split(en_test_valid, vi_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(vi_train))
print(len(en_valid))
print(len(vi_valid))
print(len(en_test))
print(len(vi_test))
print(en_train[20])
print(vi_train[20])
print(en_valid[-1])
print(vi_valid[-1])
print(en_test[-1])
print(vi_test[-1])

803200
803200
100400
100400
100400
100400
Childs?

Childs?

It's powered up now, isn't it?

Bây giờ nó bật rồi phải không nhỉ?

Why the hell are you going to Cuddy's baby naming?

Bị làm sao mà anh lại đi dự lễ đặt tên em bé của Cuddy chứ?



In [7]:
file_variable = [en_train,vi_train,en_valid,vi_valid,en_test,vi_test]
file_names = ["en_train","vi_train","en_valid","vi_valid","en_test","vi_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item.strip() + "\n")
!ls -al

total 105176
drwxrwxr-x 3 chinh chinh     4096 Thg 5  17 21:26 .
drwxrwxr-x 8 chinh chinh     4096 Thg 5  17 21:25 ..
drwxrwxr-x 2 chinh chinh     4096 Thg 5  17 21:26 data_bin
-rw-rw-r-- 1 chinh chinh  3318349 Thg 5  17 21:26 en_test
-rw-rw-r-- 1 chinh chinh 26563375 Thg 5  17 21:26 en_train
-rw-rw-r-- 1 chinh chinh  3328557 Thg 5  17 21:26 en_valid
-rw-rw-r-- 1 chinh chinh 30690851 Thg 5  17 21:26 opus-100-corpus.zip
-rw-rw-r-- 1 chinh chinh  4365722 Thg 5  17 21:26 vi_test
-rw-rw-r-- 1 chinh chinh 35019161 Thg 5  17 21:26 vi_train
-rw-rw-r-- 1 chinh chinh  4382771 Thg 5  17 21:26 vi_valid


In [8]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-bert-large')

In [9]:
start = time.time()
srcfile = "en_train"
tarfile = "vi_train"
lst_srcfile = en_train
lst_tarfile = vi_train

vec_lst_srcfile = model.encode(lst_srcfile)
print('=== Ending, Total time (second): ', time.time() - start)

=== Ending, Total time (second):  4212.303942680359


# Calculate similarity

In [10]:
import faiss
vec_lst_srcfile_copy = vec_lst_srcfile.copy()
start = time.time()
index = faiss.IndexFlatIP(vec_lst_srcfile_copy.shape[1])
index.ntotal
faiss.normalize_L2(vec_lst_srcfile_copy)
index.add(vec_lst_srcfile_copy)
k = 6
distance, index = index.search(vec_lst_srcfile_copy, k)

print('=== Ending, Total time (second): ', time.time() - start)
print(len(distance))
print(len(index))

print(index)
print(distance)

=== Ending, Total time (second):  3492.150864839554
803200
803200
[[178919 236179 534188 233242      0 157897]
 [     1 364452 144270 178997 466705   7736]
 [ 87166      2 499994 342496 513611 372074]
 ...
 [803197 680232 193396 231625 117066 726611]
 [527692 180813 803198 768526 620014 591275]
 [803199 112858 233037 547484 412678 603907]]
[[1.0000002  1.0000002  1.0000002  1.0000002  1.0000002  1.0000001 ]
 [1.0000001  0.6222546  0.60958266 0.6065708  0.6065707  0.6017508 ]
 [0.99999994 0.9999999  0.977881   0.977881   0.977881   0.977881  ]
 ...
 [0.99999994 0.99999994 0.99999994 0.9999999  0.94061303 0.92287505]
 [1.0000001  1.         1.         0.9915742  0.9915742  0.9847001 ]
 [1.         0.99999994 0.897379   0.87898946 0.8769964  0.87699634]]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance as dt

count = 0
for i in range(len(lst_srcfile)):
    j = 1 #1 is index of biggest simmilarity
    score = distance[i][j]
    a = vec_lst_srcfile[i]
    b = vec_lst_srcfile[index[i][j]]
    
    dot_product = np.dot(a, b) # x.y
    norm_a = np.linalg.norm(a) #|x|
    norm_b = np.linalg.norm(b) #|y|
    sim_res = dot_product / (norm_a * norm_b)

    if score >=0.98 :
        count+=1
        print("==========================")
        for m in range(len(index[i])):
            print("--------", distance[i][m])
        print(i, index[i][j], distance[i][j], sim_res, 1-dt.cosine(a,b))
        print(lst_srcfile[i])
        print(lst_srcfile[index[i][j]])
        print("==========================")
print(count)

In [12]:
start = time.time()
saperate = " || "

src_label = "S"
emb_label = "E"
thres_arr = [0.8, 0.85, 0.9, 0.95]
for i in range(len(thres_arr)):
    threshold = thres_arr[i]
    out_file_write = open("en_train_EM_"+str(thres_arr[i]), 'w')
    out_file_write_score = open("en_train_EM_score_"+str(thres_arr[i]), 'w')
    out_file_write_factor = open("en_train_EM_factor_"+str(thres_arr[i]), 'w')
    num_gth_thres = 0

    for i in range(len(lst_srcfile)):
        j=5
        score = distance[i][j]
        content = ""
        content_label_arr = []
        if score >= threshold and score <0.999:
            num_gth_thres += 1
            best_simi_index = index[i][j]
            content = lst_srcfile[i].strip() + saperate + lst_tarfile[best_simi_index].strip()

            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label.append(emb_label)
            content_label = content_label + [emb_label] * len(lst_tarfile[best_simi_index].split())

            content_label_arr = " ".join(content_label)
        else:
            content = lst_srcfile[i].strip()
            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label_arr = " ".join(content_label)
 
        out_file_write.writelines(content+ "\n")    
        out_file_write_factor.writelines(content_label_arr + "\n")
        out_file_write_score.writelines(str(score) + "\n")

    print('threshold =', threshold, "EM sentences =", num_gth_thres, "/", len(lst_srcfile), num_gth_thres*100/len(lst_srcfile), "%" )

    out_file_write.close()
    out_file_write_score.close()
    out_file_write_factor.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)

threshold = 0.8 EM sentences = 265707 / 803200 33.08105079681275 %
threshold = 0.85 EM sentences = 184747 / 803200 23.001369521912352 %
threshold = 0.9 EM sentences = 112594 / 803200 14.018177290836654 %
threshold = 0.95 EM sentences = 47783 / 803200 5.949078685258964 %
=== Ending, Total time (second):  26.469560623168945


In [13]:
print(len(lst_tarfile))

803200
